# Oanda Demo Trading Notebook

#### Packages

Normal Packages

In [1]:
import numpy as np
import pandas as pd

import yaml
import json

import time
import winsound
import collections

import warnings
warnings.filterwarnings('ignore')

Oanda Packages

In [2]:
from oandapyV20 import API
import oandapyV20.endpoints.trades as trades
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts

import oandapyV20.definitions.pricing as defpricing
import oandapyV20.endpoints.instruments as instruments

#### API Setup

Read from config file

In [12]:
config_file = 'config/access_token.yaml'

with open(config_file) as c_file:
    config = yaml.load(c_file)

access_token = config['oanda_demo_account']['token']
accountID = config['oanda_demo_account']['account_id']

api = API(access_token = access_token)

#### Retrive Data

In [5]:
# Get timestamp of the price and segregate it
def get_date_time(resp):
    
    time_stamp = resp['time']
    date_val, full_time = time_stamp.split(sep = 'T')
    time_val, time_fraction = full_time.split(sep = '.')
    
    return(date_val, time_val, time_fraction)

In [6]:
# Get bid and ask prices
def get_prices(resp):
    
    bid_price = float(resp['bids'][0]['price'])    
    ask_price = float(resp['asks'][0]['price'])
    spread = ask_price - bid_price
    
    return(bid_price, ask_price, spread)

In [7]:
# Terminate connection
def terminate_connection():
    try:
        print('Connection Terminated')
        r.terminate(message = "maxrecs records received")
    except:
        pass

In [8]:
def get_chunks(i, val):
    global chunk_size
    global small_list
    global size_flag
    global change_position
    
    if i < chunk_size: #Keep adding prices until we reach chunk size
        small_list.append(val)  

    if len(small_list) == chunk_size: #check if small list has reached chunk size
        size_flag = 1          
        
    if size_flag == 1:
        small_list.popleft()   #Remove the left most (first) price
        small_list.append(val) #Add current value to right most position
        
        change_position -= 1  #Price allocation in small list happens right to left so change position goes from chunk size to zero
        if change_position < 0: 
            change_position = chunk_size - 1 #If position reaches zero, it needs to be reset to chunk size - 1 (for index position)
               
    return(small_list)

In [9]:
def find_direction(lst):
    list_size = len(lst)
    val_dict = {}
    flag_val = 0

    for i in range(list_size): #write the list into a dictionary 
        val_dict[i] = lst[i]


    for i in range(list_size-1):
        if val_dict[i] < val_dict[i+1]:
            flag_val += 1

        elif val_dict[i] == val_dict[i+1]:
            flag_val += 0

        elif val_dict[i] > val_dict[i+1]:
            flag_val -= 1

    if flag_val == len(lst) - 1:
        direction = 'positive'
    elif flag_val == (len(lst) - 1)*-1:
        direction = 'negative'
    else:
        direction = 'no direction'
        
    return(flag_val, direction)

In [10]:
full_list = [1,2,3,4,5,6,7,8,9,10,11,12,12,11,10,9,5,6,17,25,20,19,18]

chunk_size = 3
change_position = chunk_size - 1
small_list = collections.deque([])
size_flag = 0

for i, val in enumerate(full_list):
    
    small_list = get_chunks(i, val)
    flag_val, direction = find_direction(small_list)
        
    print(small_list, flag_val, direction)

deque([1]) 0 positive
deque([1, 2]) 1 positive
deque([2, 3, 3]) 1 no direction
deque([3, 3, 4]) 1 no direction
deque([3, 4, 5]) 2 positive
deque([4, 5, 6]) 2 positive
deque([5, 6, 7]) 2 positive
deque([6, 7, 8]) 2 positive
deque([7, 8, 9]) 2 positive
deque([8, 9, 10]) 2 positive
deque([9, 10, 11]) 2 positive
deque([10, 11, 12]) 2 positive
deque([11, 12, 12]) 1 no direction
deque([12, 12, 11]) -1 no direction
deque([12, 11, 10]) -2 negative
deque([11, 10, 9]) -2 negative
deque([10, 9, 5]) -2 negative
deque([9, 5, 6]) 0 no direction
deque([5, 6, 17]) 2 positive
deque([6, 17, 25]) 2 positive
deque([17, 25, 20]) 0 no direction
deque([25, 20, 19]) -2 negative
deque([20, 19, 18]) -2 negative


In [11]:
r = pricing.PricingStream(accountID=accountID, params=params)
rv = api.request(r)

num = 4
bp_list = []
sp_list = []

for i, resp in enumerate(rv):
    if i < num:        
        print('Iteration : ',i)
        
        resp_type = resp['type']        
        
        if resp_type == 'PRICE':    
            date_val, time_val, time_fraction = get_date_time(resp)
            sell_price, buy_price, spread = get_prices(resp)          
            print('buy_price:',buy_price,',' , 'sell_price:',sell_price, ',', 'spread(pips):',spread*10000)
            print('----------------------------------------------')
            
        else:
            print('HEARTBEAT')

    else:
        terminate_connection()
        
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Iteration :  0
buy_price: 1.12925 , sell_price: 1.12906 , spread(pips): 1.900000000001345
----------------------------------------------
Iteration :  1
HEARTBEAT
Iteration :  2
HEARTBEAT
Iteration :  3
HEARTBEAT
Connection Terminated


# 2. fit into full code

In [21]:
params = {
          "instrument": "EUR_USD",
          "period": -3600
        }
        
r = instruments.InstrumentsPositionBook(instrument="EUR_USD", params=params)
api.request(r)
print(json.dumps(r.response,indent = 2))

{
  "positionBook": {
    "instrument": "EUR_USD",
    "time": "2020-06-06T16:20:00Z",
    "unixTime": "1591460400",
    "price": "1.12916",
    "bucketWidth": "0.00050",
    "buckets": [
      {
        "price": "1.03500",
        "longCountPercent": "0.0000",
        "shortCountPercent": "0.0016"
      },
      {
        "price": "1.03600",
        "longCountPercent": "0.0000",
        "shortCountPercent": "0.0016"
      },
      {
        "price": "1.03650",
        "longCountPercent": "0.0016",
        "shortCountPercent": "0.0016"
      },
      {
        "price": "1.03700",
        "longCountPercent": "0.0016",
        "shortCountPercent": "0.0032"
      },
      {
        "price": "1.03750",
        "longCountPercent": "0.0000",
        "shortCountPercent": "0.0048"
      },
      {
        "price": "1.03800",
        "longCountPercent": "0.0048",
        "shortCountPercent": "0.0065"
      },
      {
        "price": "1.03850",
        "longCountPercent": "0.0065",
        "sho